In [ ]:
"""
News Bias Classification System - Open Source Implementation
Two-Stage Hybrid Architecture: Hugging Face + Llama 3

Stage 1: Binary screening (Local BERT model - FREE)
Stage 2: Subtype reasoning (Llama 3 via HF API - Cost-effective)

Author: Migrated from OpenAI to open-source solution
Dataset: MBIC (Media Bias Annotation by Crowdsourcing and Experts)
Goal: Multi-label bias classification with explainability
"""


'\nNews Bias Classification System - Open Source Implementation\nTwo-Stage Hybrid Architecture: Hugging Face + Llama 3\n\nStage 1: Binary screening (Local BERT model - FREE)\nStage 2: Subtype reasoning (Llama 3 via HF API - Cost-effective)\n\nAuthor: Migrated from OpenAI to open-source solution\nDataset: MBIC (Media Bias Annotation by Crowdsourcing and Experts)\nGoal: Multi-label bias classification with explainability\n'

In [ ]:

import pandas as pd
import numpy as np
import json
import os
import requests
from typing import List, Dict, Tuple, Optional
from dataclasses import dataclass, field
from sklearn.model_selection import train_test_split
from sklearn.metrics import (f1_score, precision_score, recall_score,
                            classification_report, confusion_matrix)
from tqdm.auto import tqdm
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os

def setup_colab_environment():
    print("DEBUG: Function started")  # <-- Does this show?

    try:
        import google.colab
        print("DEBUG: google.colab imported")  # test
        IN_COLAB = True
        print("🔵 Running in Google Colab")

        print("\n📦 Installing required packages...")
        os.system('pip install -q transformers torch accelerate requests lime openpyxl')

    except Exception as e:
        print("ERROR inside function:", e)   # show errors clearly
        IN_COLAB = False
        print("🔵 Running in local environment")

    return IN_COLAB

setup_colab_environment()


DEBUG: Function started
DEBUG: google.colab imported
🔵 Running in Google Colab

📦 Installing required packages...


True

In [ ]:
# ============================================================================
# PROJECT CONFIGURATION (UPDATED)
# ============================================================================

import os
from dataclasses import dataclass, field
from typing import List

@dataclass
class ProjectConfig:
    """
    Centralized configuration for the MBIC Bias Classification pipeline.
    Includes dataset split ratios, evaluation thresholds, API settings, and
    secure Hugging Face token handling.
    """

    # --- API Token (SAFE: pulled from environment variable) ---
    HUGGINGFACE_TOKEN: str = field(default_factory=lambda: os.getenv("HF_TOKEN", None))

    # --- Two-Stage Model Architecture ---
    BINARY_MODEL: str = "newsmediabias/UnBIAS-classifier"           # Stage 1 (local)
    SUBTYPE_MODEL: str = "meta-llama/Meta-Llama-3-8B-Instruct"      # Stage 2 (API)
    INFERENCE_API_URL: str = "https://api-inference.huggingface.co/models/"

    # --- Dataset + Output Paths ---
    MBIC_FILEPATH: str = "/content/Copy of labeled_dataset.xlsx"
    OUTPUT_DIR: str = "bias_classification_outputs"

    # --- Bias Subtypes ---
    BIAS_SUBTYPES: List[str] = field(default_factory=lambda: [
        "Political Bias",
        "Linguistic Bias",
        "Racial Bias",
        "Gender Bias",
        "Text-level Context",
        "Reporting-level Context",
        "Cognitive Bias",
        "Fake News",
        "Hate Speech"
    ])

    # --- Dataset Split Ratios ---
    TRAIN_RATIO: float = 0.70
    VAL_RATIO: float = 0.10
    TEST_RATIO: float = 0.20

    # --- Evaluation Settings ---
    HUMAN_VALIDATION_SAMPLE_SIZE: int = 100
    F1_THRESHOLD: float = 0.80

    # --- API Settings ---
    MAX_API_RETRIES: int = 3
    API_TIMEOUT: int = 30

    def __post_init__(self):
        """Create output directory and validate token & split ratios."""
        # Create output directory if missing
        os.makedirs(self.OUTPUT_DIR, exist_ok=True)

        # Hugging Face token check
        if not self.HUGGINGFACE_TOKEN:
            print("⚠️ WARNING: No Hugging Face token found. Set HF_TOKEN environment variable or use set_hf_token()")
        else:
            print("🔑 Hugging Face token loaded successfully.")

        # Validate dataset split ratios sum to 1.0
        total_ratio = self.TRAIN_RATIO + self.VAL_RATIO + self.TEST_RATIO
        if not abs(total_ratio - 1.0) < 1e-6:
            print(f"⚠️ Warning: Dataset split ratios sum to {total_ratio}, not 1.0")

    def set_hf_token(self, token: str):
        """Set or update Hugging Face API token"""
        self.HUGGINGFACE_TOKEN = token
        os.environ['HF_TOKEN'] = token
        print("🔑 Hugging Face token updated successfully.")


In [ ]:


    # Dataset split ratios
    TRAIN_RATIO: float = 0.70
    VAL_RATIO: float = 0.10
    TEST_RATIO: float = 0.20

    # Evaluation settings
    HUMAN_VALIDATION_SAMPLE_SIZE: int = 100
    F1_THRESHOLD: float = 0.80

    # API settings
    MAX_API_RETRIES: int = 3
    API_TIMEOUT: int = 30

    def __post_init__(self):
        os.makedirs(self.OUTPUT_DIR, exist_ok=True)

    def set_hf_token(self, token: str):
        """Set Hugging Face API token"""
        self.HUGGINGFACE_TOKEN = token
        os.environ['HUGGINGFACE_TOKEN'] = token


# Comprehensive bias definitions for LLM prompting
BIAS_DEFINITIONS = {
    "Political Bias": """
    Favoring or opposing specific political parties, ideologies, or figures.
    Examples: Partisan language, selective coverage of political events,
    framing issues to benefit one political side.
    """,

    "Linguistic Bias": """
    Word choice that influences perception through loaded language, euphemisms,
    or emotionally charged terms. Examples: "freedom fighter" vs "terrorist",
    "tax relief" vs "tax cuts", "illegal aliens" vs "undocumented immigrants".
    """,

    "Racial Bias": """
    Stereotypes, prejudice, or discrimination based on race or ethnicity.
    Examples: Racial profiling, stereotypical associations,
    disproportionate negative coverage of specific racial groups.
    """,

    "Gender Bias": """
    Stereotypes or discrimination based on gender, including sexist language
    or gendered expectations. Examples: "Bossy" for women vs "assertive" for men,
    focus on appearance for women but competence for men.
    """,

    "Text-level Context": """
    Misleading framing or manipulation of context within the text itself.
    Examples: Cherry-picking quotes, juxtaposing unrelated facts to create
    false implications, misleading headlines.
    """,

    "Reporting-level Context": """
    Selective reporting, omission of important facts, or one-sided coverage.
    Examples: Covering only negative aspects of a topic, ignoring counterarguments,
    unbalanced source selection.
    """,

    "Cognitive Bias": """
    Logical fallacies or manipulative reasoning patterns that exploit
    psychological biases. Examples: False dilemmas, slippery slope arguments,
    appeal to emotion, confirmation bias reinforcement.
    """,

    "Fake News": """
    Demonstrably false information presented as fact, fabricated content,
    or deliberately misleading claims. Examples: Made-up statistics,
    false attributions, doctored quotes, conspiracy theories.
    """,

    "Hate Speech": """
    Hostile, threatening, or discriminatory language targeting individuals
    or groups based on protected characteristics. Examples: Slurs,
    dehumanizing language, incitement to violence or discrimination.
    """
}


# ============================================================================
# PHASE I: DATA LOADING & PREPARATION
# ============================================================================

class MBICDataLoader:
    """Load and prepare MBIC dataset - handles Excel, CSV, and JSON"""

    def __init__(self, filepath: str):
        self.filepath = filepath
        self.df = None
        self.df_filtered = None

    def load_data(self) -> pd.DataFrame:
        """Load MBIC dataset - handles Excel, CSV, and JSON formats"""
        file_ext = os.path.splitext(self.filepath)[1].lower()

        try:
            if file_ext in ['.xlsx', '.xls']:
                # Load Excel file
                print(f"📂 Loading Excel file: {self.filepath}")
                self.df = pd.read_excel(self.filepath)
                print(f"✓ Loaded Excel: {len(self.df)} records")

            elif file_ext == '.csv':
                # Load CSV file
                print(f"📂 Loading CSV file: {self.filepath}")
                self.df = pd.read_csv(self.filepath)
                print(f"✓ Loaded CSV: {len(self.df)} records")

            elif file_ext == '.json':
                # Load JSON file
                print(f"📂 Loading JSON file: {self.filepath}")
                self.df = pd.read_json(self.filepath, lines=True)
                print(f"✓ Loaded JSON: {len(self.df)} records")

            else:
                raise ValueError(f"Unsupported file format: {file_ext}")

        except Exception as e:
            raise ValueError(f"Could not load dataset: {e}")

        print(f"  Columns: {list(self.df.columns)}")
        self._standardize_columns()
        return self.df

    def _standardize_columns(self):
        """Standardize column names across different formats"""
        column_mapping = {
            'sentence': 'Sentence_Text',
            'text': 'Sentence_Text',
            'sentence_text': 'Sentence_Text',
            'content': 'Sentence_Text',
            'article': 'Sentence_Text',
            'label': 'Binary_Label',
            'bias_label': 'Binary_Label',
            'bias': 'Binary_Label',
            'classification': 'Binary_Label',
            'id': 'sentence_id',
            'sent_id': 'sentence_id',
            'article_id': 'sentence_id'
        }

        for old_col, new_col in column_mapping.items():
            if old_col in self.df.columns and new_col not in self.df.columns:
                self.df.rename(columns={old_col: new_col}, inplace=True)

        if 'sentence_id' not in self.df.columns:
            self.df['sentence_id'] = [f"sent_{i:04d}" for i in range(len(self.df))]

        # Handle Binary_Label if missing
        if 'Binary_Label' not in self.df.columns:
            print("⚠️  Warning: No Binary_Label column found. All sentences will be processed.")
            self.df['Binary_Label'] = 'Unknown'

    def filter_no_agreement(self) -> pd.DataFrame:
        """Filter out 'No Agreement' cases"""
        if 'Binary_Label' not in self.df.columns:
            print("⚠️  No Binary_Label column - keeping all records")
            self.df_filtered = self.df.copy()
        else:
            self.df_filtered = self.df[
                self.df['Binary_Label'] != 'No Agreement'
            ].copy()

        print(f"\n=== Phase I.1: MBIC Preparation ===")
        print(f"Original size: {len(self.df)}")
        print(f"After filtering 'No Agreement': {len(self.df_filtered)}")

        return self.df_filtered

    def prepare_for_annotation(self) -> pd.DataFrame:
        """Full pipeline: load → filter"""
        self.load_data()
        self.filter_no_agreement()
        return self.df_filtered


# ============================================================================
# PHASE I.2: SILVER LABEL GENERATION (TWO-STAGE ARCHITECTURE)
# ============================================================================

class SilverLabelAnnotator:
    """
    Two-Stage Hybrid Annotator:
    Stage 1: Binary classification (Local BERT - FREE)
    Stage 2: Subtype reasoning (Llama 3 API - Only for biased)
    """

    def __init__(self, config: ProjectConfig):
        self.config = config
        self.bias_types = config.BIAS_SUBTYPES
        self.annotation_log = []

        # Stage 1: Initialize local binary classifier
        print("\n🔄 Loading Stage 1: Binary Classifier (Local)...")
        self._initialize_binary_classifier()

        # Stage 2: Setup API headers for Llama 3
        self.api_headers = {
            "Authorization": f"Bearer {config.HUGGINGFACE_TOKEN}"
        }
        self.api_url = f"{config.INFERENCE_API_URL}{config.SUBTYPE_MODEL}"

        print(f"✅ Two-stage annotator ready")
        print(f"   Stage 1: {config.BINARY_MODEL}")
        print(f"   Stage 2: {config.SUBTYPE_MODEL}")

    def _initialize_binary_classifier(self):
        """Load local binary classifier (no API needed)"""
        try:
            from transformers import pipeline

            # Load the UnBIAS binary classifier
            self.binary_classifier = pipeline(
                "text-classification",
                model=self.config.BINARY_MODEL,
                device=-1  # CPU (use 0 for GPU)
            )
            print(f"   ✓ Loaded: {self.config.BINARY_MODEL}")

        except Exception as e:
            raise RuntimeError(f"Failed to load binary classifier: {e}")

    def _stage1_binary_check(self, sentence: str) -> Dict:
        """
        Stage 1: Fast local binary classification
        Returns: {'is_biased': bool, 'confidence': float}
        """
        try:
            result = self.binary_classifier(sentence, truncation=True, max_length=512)[0]

            # Map model output to binary decision
            label = result['label'].lower()
            is_biased = 'bias' in label or label == 'biased'

            return {
                'is_biased': is_biased,
                'confidence': float(result['score']),
                'raw_label': result['label']
            }

        except Exception as e:
            # Default to biased if error (to ensure Stage 2 review)
            return {
                'is_biased': True,
                'confidence': 0.5,
                'error': str(e)
            }

    def _create_llama3_prompt(self, sentence: str, sentence_id: str) -> str:
        """Create prompt for Llama 3 subtype reasoning"""
        definitions_text = "\n\n".join([
            f"{i+1}. **{bias_type}**\n{BIAS_DEFINITIONS[bias_type].strip()}"
            for i, bias_type in enumerate(self.bias_types)
        ])

        prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are an expert media bias annotator. Your task is to classify news sentences into specific bias categories.<|eot_id|><|start_header_id|>user<|end_header_id|>

========== BIAS TYPE DEFINITIONS ==========

{definitions_text}

========== CLASSIFICATION TASK ==========

Sentence ID: {sentence_id}
Sentence: "{sentence}"

INSTRUCTIONS:
1. This sentence has already been flagged as containing bias
2. Identify which of the 9 bias types is MOST PROMINENT
3. Provide a confidence score (0.0 to 1.0)
4. Briefly explain the bias

Return ONLY valid JSON with this exact structure:
{{
  "bias_type": "<one of the 9 types above>",
  "confidence_score": 0.85,
  "bias_description": "<brief explanation>"
}}

CRITICAL: bias_type must be EXACTLY one of: {', '.join(self.bias_types)}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""
        return prompt

    def _stage2_subtype_reasoning(self, sentence: str, sentence_id: str,
                                  max_retries: int = None) -> Dict:
        """
        Stage 2: Call Llama 3 via Hugging Face Inference API
        Returns: {'bias_type': str, 'confidence_score': float, 'bias_description': str}
        """
        if max_retries is None:
            max_retries = self.config.MAX_API_RETRIES

        prompt = self._create_llama3_prompt(sentence, sentence_id)

        payload = {
            "inputs": prompt,
            "parameters": {
                "max_new_tokens": 200,
                "temperature": 0.3,
                "top_p": 0.9,
                "return_full_text": False
            }
        }

        for attempt in range(max_retries):
            try:
                response = requests.post(
                    self.api_url,
                    headers=self.api_headers,
                    json=payload,
                    timeout=self.config.API_TIMEOUT
                )

                if response.status_code == 503:
                    # Model is loading, wait and retry
                    wait_time = 2 ** attempt
                    print(f"   ⏳ Model loading, waiting {wait_time}s...")
                    time.sleep(wait_time)
                    continue

                response.raise_for_status()

                # Parse response
                result = response.json()

                if isinstance(result, list) and len(result) > 0:
                    content = result[0].get('generated_text', '')
                else:
                    content = result.get('generated_text', '')

                # Extract JSON from response
                content = content.strip()
                if "```json" in content:
                    content = content.split("```json")[1].split("```")[0].strip()
                elif "```" in content:
                    content = content.split("```")[1].split("```")[0].strip()

                # Find JSON object
                if '{' in content:
                    start = content.index('{')
                    end = content.rindex('}') + 1
                    content = content[start:end]

                parsed = json.loads(content)

                # Validation
                valid_types = self.bias_types
                if parsed.get('bias_type') not in valid_types:
                    # Try to match partial
                    for valid_type in valid_types:
                        if valid_type.lower() in parsed.get('bias_type', '').lower():
                            parsed['bias_type'] = valid_type
                            break
                    else:
                        parsed['bias_type'] = self.bias_types[0]  # Default

                confidence = parsed.get('confidence_score', 0.7)
                if not (0.0 <= confidence <= 1.0):
                    confidence = 0.7
                parsed['confidence_score'] = confidence

                return {
                    'bias_type': parsed['bias_type'],
                    'confidence_score': confidence,
                    'bias_description': parsed.get('bias_description', '')
                }

            except requests.exceptions.Timeout:
                if attempt == max_retries - 1:
                    return {
                        'bias_type': 'ERROR',
                        'confidence_score': 0.0,
                        'bias_description': 'API timeout'
                    }
                time.sleep(2 ** attempt)

            except Exception as e:
                if attempt == max_retries - 1:
                    return {
                        'bias_type': 'ERROR',
                        'confidence_score': 0.0,
                        'bias_description': f'Error: {str(e)}'
                    }
                time.sleep(2 ** attempt)

        return {
            'bias_type': 'ERROR',
            'confidence_score': 0.0,
            'bias_description': 'Max retries exceeded'
        }

    def annotate_single_sentence(self, sentence: str, sentence_id: str) -> Dict:
        """
        TWO-STAGE ANNOTATION PIPELINE
        Stage 1: Binary check (local, fast, free)
        Stage 2: Subtype reasoning (API, only if biased)
        """
        # STAGE 1: Binary classification (local)
        stage1_result = self._stage1_binary_check(sentence)

        # If neutral, skip Stage 2 entirely
        if not stage1_result['is_biased']:
            self.annotation_log.append({
                'sentence_id': sentence_id,
                'status': 'neutral_skipped',
                'stage1_confidence': stage1_result['confidence']
            })

            return {
                'sentence_id': sentence_id,
                'bias_type': 'None',
                'confidence_score': stage1_result['confidence'],
                'bias_description': 'Neutral/objective text',
                'stage1_decision': 'neutral'
            }

        # STAGE 2: Subtype reasoning (API, only for biased)
        stage2_result = self._stage2_subtype_reasoning(sentence, sentence_id)

        self.annotation_log.append({
            'sentence_id': sentence_id,
            'status': 'biased_annotated',
            'stage1_confidence': stage1_result['confidence'],
            'stage2_type': stage2_result['bias_type']
        })

        return {
            'sentence_id': sentence_id,
            'bias_type': stage2_result['bias_type'],
            'confidence_score': stage2_result['confidence_score'],
            'bias_description': stage2_result['bias_description'],
            'stage1_decision': 'biased',
            'stage1_confidence': stage1_result['confidence']
        }

    def annotate_dataset(self, df: pd.DataFrame,
                        sentence_col: str = 'Sentence_Text',
                        id_col: str = 'sentence_id',
                        save_checkpoints: bool = True) -> pd.DataFrame:
        """
        Annotate entire dataset with two-stage pipeline
        """
        print(f"\n=== Phase I.2: Two-Stage Silver Label Generation ===")
        print(f"🎯 Processing {len(df)} sentences")
        print(f"📊 Stage 1: {self.config.BINARY_MODEL} (Local)")
        print(f"📊 Stage 2: {self.config.SUBTYPE_MODEL} (API)")

        # Estimate biased sentences if Binary_Label exists
        if 'Binary_Label' in df.columns and df['Binary_Label'].notna().any():
            biased_estimate = len(df[df['Binary_Label'] == 'Biased'])
            print(f"⏱️  Estimated API calls: ~{biased_estimate}\n")
        else:
            print(f"⏱️  Estimated API calls: TBD (will filter with Stage 1)\n")

        results = []
        checkpoint_interval = 50

        stage1_count = 0
        stage2_count = 0

        for idx, row in tqdm(df.iterrows(), total=len(df),
                            desc="Two-Stage Annotation"):
            sentence = str(row[sentence_col])
            sent_id = str(row[id_col])

            result = self.annotate_single_sentence(sentence, sent_id)
            results.append(result)

            # Track stage usage
            if result['stage1_decision'] == 'neutral':
                stage1_count += 1
            else:
                stage2_count += 1

            # Checkpoint saving
            if save_checkpoints and (idx + 1) % checkpoint_interval == 0:
                checkpoint_df = pd.DataFrame(results)
                checkpoint_path = os.path.join(
                    self.config.OUTPUT_DIR,
                    f'checkpoint_silver_{idx+1}.csv'
                )
                checkpoint_df.to_csv(checkpoint_path, index=False)
                print(f"\n💾 Checkpoint: {checkpoint_path}")
                print(f"   Stage 1 only: {stage1_count}, Stage 2 calls: {stage2_count}")

            # Small delay only for API calls
            if result['stage1_decision'] == 'biased':
                time.sleep(0.5)

        # Create results DataFrame
        annotations_df = pd.DataFrame(results)

        # Merge with original data
        df_annotated = df.merge(
            annotations_df,
            left_on=id_col,
            right_on='sentence_id',
            how='left',
            suffixes=('', '_annotation')
        )

        df_annotated['Subtype_Silver_Label'] = df_annotated['bias_type']
        df_annotated['Silver_Confidence'] = df_annotated['confidence_score']

        # Save final annotations
        final_path = os.path.join(self.config.OUTPUT_DIR, 'silver_labels.csv')
        df_annotated.to_csv(final_path, index=False)

        self._print_summary(df_annotated, stage1_count, stage2_count)

        return df_annotated

    def _print_summary(self, df: pd.DataFrame, stage1_count: int, stage2_count: int):
        """Print detailed annotation statistics"""
        print("\n" + "="*70)
        print("TWO-STAGE SILVER LABEL GENERATION COMPLETE")
        print("="*70)

        print(f"\n📊 Pipeline Statistics:")
        print(f"   Total processed: {len(df)}")
        print(f"   Stage 1 only (Neutral): {stage1_count} ({stage1_count/len(df)*100:.1f}%)")
        print(f"   Stage 2 API calls (Biased): {stage2_count} ({stage2_count/len(df)*100:.1f}%)")
        print(f"   💰 Cost savings: {stage1_count/len(df)*100:.1f}% of sentences skipped Stage 2")

        error_count = len(df[df['Subtype_Silver_Label'] == 'ERROR'])
        print(f"\n   Errors: {error_count} ({error_count/len(df)*100:.2f}%)")

        print("\n📊 Bias Type Distribution:")
        dist = df['Subtype_Silver_Label'].value_counts()
        for bias_type, count in dist.items():
            pct = count / len(df) * 100
            print(f"   {bias_type:30s}: {count:4d} ({pct:5.2f}%)")

        valid_conf = df[df['Subtype_Silver_Label'] != 'ERROR']['Silver_Confidence']
        if len(valid_conf) > 0:
            print(f"\n📈 Confidence Statistics:")
            print(f"   Mean:   {valid_conf.mean():.3f}")
            print(f"   Median: {valid_conf.median():.3f}")
            print(f"   Std:    {valid_conf.std():.3f}")


# ============================================================================
# PHASE I.3-I.5: MASTER DATASET & SPLITS
# ============================================================================

class DatasetBuilder:
    """Create master dataset and train/val/test splits"""

    def __init__(self, config: ProjectConfig):
        self.config = config

    def create_master_dataset(self, df_annotated: pd.DataFrame) -> pd.DataFrame:
        """Create master dataset from annotated data"""
        print(f"\n=== Phase I.4: Create Master Dataset ===")

        master_df = df_annotated.copy()

        # Save master dataset
        output_path = os.path.join(self.config.OUTPUT_DIR, 'master_dataset.csv')
        master_df.to_csv(output_path, index=False)

        print(f"✓ Master dataset: {len(master_df)} sentences")
        print(f"  Saved: {output_path}")

        return master_df

    def create_splits(self, df: pd.DataFrame) -> Tuple[pd.DataFrame, ...]:
        """Create stratified train/val/test splits"""
        print(f"\n=== Phase I.5: Data Split ===")

        # Remove errors
        df_clean = df[df['Subtype_Silver_Label'] != 'ERROR'].copy()

        # Check if we have enough data per class
        class_counts = df_clean['Subtype_Silver_Label'].value_counts()
        min_class_count = class_counts.min()

        if min_class_count < 2:
            print(f"⚠️  Warning: Some classes have < 2 samples. Skipping stratification.")
            # Non-stratified split
            train_df, temp_df = train_test_split(
                df_clean,
                test_size=(self.config.VAL_RATIO + self.config.TEST_RATIO),
                random_state=42
            )

            val_ratio = self.config.VAL_RATIO / (
                self.config.VAL_RATIO + self.config.TEST_RATIO
            )
            val_df, test_df = train_test_split(
                temp_df,
                test_size=(1 - val_ratio),
                random_state=42
            )
        else:
            # Stratified split
            train_df, temp_df = train_test_split(
                df_clean,
                test_size=(self.config.VAL_RATIO + self.config.TEST_RATIO),
                stratify=df_clean['Subtype_Silver_Label'],
                random_state=42
            )

            val_ratio = self.config.VAL_RATIO / (
                self.config.VAL_RATIO + self.config.TEST_RATIO
            )
            val_df, test_df = train_test_split(
                temp_df,
                test_size=(1 - val_ratio),
                stratify=temp_df['Subtype_Silver_Label'],
                random_state=42
            )

        total = len(df_clean)
        print(f"\n📊 Dataset Split:")
        print(f"  Total:  {total:5d}")
        print(f"  Train:  {len(train_df):5d} ({len(train_df)/total*100:5.1f}%)")
        print(f"  Val:    {len(val_df):5d} ({len(val_df)/total*100:5.1f}%)")
        print(f"  Test:   {len(test_df):5d} ({len(test_df)/total*100:5.1f}%)")

        # Save splits
        for name, split_df in [('train', train_df), ('val', val_df), ('test', test_df)]:
            path = os.path.join(self.config.OUTPUT_DIR, f'{name}_split.csv')
            split_df.to_csv(path, index=False)
            print(f"  ✓ {name}_split.csv")

        return train_df, val_df, test_df


# ============================================================================
# PHASE III: EVALUATION
# ============================================================================

class BiasEvaluator:
    """Comprehensive evaluation metrics"""

    def __init__(self, config: ProjectConfig):
        self.config = config

    def evaluate_multiclass(self, y_true: List[str], y_pred: List[str]) -> Dict:
        """Multi-class classification report"""
        print("\n=== Phase III: Multi-Class Evaluation ===")

        class_names = self.config.BIAS_SUBTYPES + ['None']

        report = classification_report(
            y_true, y_pred,
            target_names=class_names,
            output_dict=True,
            zero_division=0
        )

        print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))

        return report

    def export_human_validation_sample(self, df: pd.DataFrame) -> pd.DataFrame:
        """Export sample for manual review"""
        print(f"\n=== Human Validation Sample ===")

        n = min(self.config.HUMAN_VALIDATION_SAMPLE_SIZE, len(df))

        sample_df = df.sample(n=n, random_state=42).copy()

        sample_df['Judge_1'] = ''
        sample_df['Judge_2'] = ''
        sample_df['Judge_3'] = ''
        sample_df['Consensus'] = ''
        sample_df['Notes'] = ''

        output_path = os.path.join(self.config.OUTPUT_DIR, 'human_validation.csv')
        sample_df.to_csv(output_path, index=False)

        print(f"✓ Sampled {len(sample_df)} sentences")
        print(f"  Saved: {output_path}")
        print("\nDistribution:")
        print(